# Signe Language Recognition 

Lucas Piquet / Henri Vasseur 

<h1><a id='toc'></a>Sommaire</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#0">0.  MediaPipe's functions </a></li>
        <li><a href="#I">1. Data Collection </a></li>
        <li><a href="#II">2.  Data Selection / Validity of acquisition</a></li> 
        <li><a href="#III">3. Model Training</a></li>
        <li><a href="#IV">4. Real time test</a></li> 
    </ul>

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from scipy import stats
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


<h2><a id='0'></a>
0.  MediaPipe's functions </h2> 

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
#
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
# 
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
#
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 


In [6]:
# Extract Keypoint Values
def extract_keypoints(results):
    """Returns:
    - numpy.ndarray:
      A 1D numpy array containing concatenated keypoint values.
      The order of keypoints is as follows:
      [left_hand_landmarks, right_hand_landmarks, pose_landmarks, face_landmarks] 
      shape = (1662,)
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh, pose, face])

<h2><a id='I'></a>
1.  Data Collection </h2> 

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data2') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou','nogesture'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 25


In [22]:
# Add new folders if needed
# Check if the directory is empty
if not DATA_PATH in os.listdir() :
    # Directory is empty, handle it as needed
    os.makedirs(os.path.join(DATA_PATH))
    # Your code to handle an empty directory goes here
for action in actions:
        
    # Directory is not empty, proceed with creating new directories
    dir_action = os.path.join(DATA_PATH, action)
    
    if not action in os.listdir(DATA_PATH) :
    # Directory is empty, handle it as needed
        os.makedirs(os.path.join(dir_action))
    # Your code to handle an empty directory goes here




In [8]:
# Folder start
start_folder = len(os.listdir(os.path.join(DATA_PATH, actions[0]))) + 1

print(start_folder)
print(actions)


61
['hello' 'thanks' 'iloveyou' 'nogesture']


In [80]:
time.sleep(1)

cap = cv2.VideoCapture(0)

pygame.mixer.init()

# Charger des sons 
start_sound = pygame.mixer.Sound("Sound/start_bip.wav")
end_sound = pygame.mixer.Sound("Sound/end_bip.wav")
action_bip = pygame.mixer.Sound("Sound/action_bip.wav")


# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos

        # Jouer un son pour différencier les actions
        pygame.mixer.Sound.play(action_bip)

        time.sleep(1)

        for sequence in range(start_folder, start_folder + no_sequences):

            # Jouer un son pour indiquer le début de l'enregistrement de la vidéo
            pygame.mixer.Sound.play(start_sound)

            # Initialize results array
            sign = np.zeros((1662,sequence_length))

            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                        
                sign[:,frame_num] = extract_keypoints(results)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
            # Jouer un son pour indiquer la fin de l'enregistrement de la vidéo
            pygame.mixer.Sound.play(end_sound)

            # Save keypoints
            npy_path = os.path.join(DATA_PATH, action, str(sequence))
            np.save(npy_path, sign)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            # Pause between videos
            time.sleep(1.25)

        if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        pygame.mixer.Sound.play(action_bip)
        time.sleep(1.25)
        
        
    # Update folder start number
    start_folder += no_sequences
    cap.release()
    cv2.destroyAllWindows()

In [9]:
DATA_PATH = os.path.join('Data_viz_test')

In [22]:
# Fast Video Capture

time.sleep(1)
cap = cv2.VideoCapture(0)
pygame.mixer.init()

# Charger un son (remplacez le chemin avec votre propre fichier audio)
start_sound = pygame.mixer.Sound("start_bip.wav")
end_sound = pygame.mixer.Sound("end_bip.wav")

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Action to capture (adjust as needed)
    action_to_capture = "hello"

    # Number of frames to capture
    num_frames_to_capture = 30

    # Create an array to store landmarks
    sign_landmarks = np.zeros((1662, num_frames_to_capture))
    
    # Jouer un son au début
    pygame.mixer.Sound.play(start_sound)

    # Capture frames
    for frame_num in range(num_frames_to_capture):
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Apply wait logic
        if frame_num == 0:
            cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames for {} Sign'.format(action_to_capture), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            cv2.waitKey(100)
        else:
            cv2.putText(image, 'Collecting frames for {} Sign'.format(action_to_capture), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

        # Store landmarks
        sign_landmarks[:,frame_num] = extract_keypoints(results)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    pygame.mixer.Sound.play(end_sound)
    # Save landmarks
    npy_path = os.path.join(DATA_PATH, action_to_capture)
    np.save(npy_path, sign_landmarks)
    

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


<h2><a id='II'></a>
2.  Data Selection / Validity of acquisition </h2> 

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
print(label_map)

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'nogesture': 3}


In [10]:
sign = np.load(os.path.join(DATA_PATH, actions[0], "1.npy"))
print(sign.shape)

(1662, 25)


In [11]:
# nombre de video dans chaque répertoire 
nbr_sign = len(os.listdir(os.path.join(DATA_PATH, actions[0])))

X = np.zeros((actions.shape[0]*nbr_sign, sign.shape[1], sign.shape[0]))
y = np.zeros((actions.shape[0]*nbr_sign, 1))

X_without_face = np.zeros((actions.shape[0]*nbr_sign, sign.shape[1], sign.shape[0]-468*3))

# ...

for action in actions:
    action_path = os.path.join(DATA_PATH, action)

    for sequence, sequence_path in enumerate(os.listdir(action_path)):
        sequence_path = os.path.join(action_path, "{}.npy".format(sequence+1))
        
        # Vérifiez si le fichier existe avant de le charger
        if os.path.exists(sequence_path):
            X[nbr_sign*label_map[action]+sequence, :, :] = np.load(sequence_path).T
            y[nbr_sign*label_map[action]+sequence,] = label_map[action]
            X_without_face[nbr_sign*label_map[action]+sequence, :, :] = np.load(sequence_path).T[:, :-468*3]
        else:
            # print(f"Le fichier {file_path} n'existe pas. Ignorer cette séquence.")
            break  # Arrête la boucle si un fichier est manquant

# ...
y = to_categorical(y).astype(int)


In [82]:
print(X.shape, X_without_face.shape,y.shape)

(240, 25, 1662) (240, 25, 258) (240, 4)


In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
y_test

array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0]])

In [12]:
X_train_hands, X_test_hands, y_train_hands, y_test_hands = train_test_split(X_without_face, y, test_size=0.2)
print(X_train_hands.shape, X_test_hands.shape, y_train_hands.shape, y_test_hands.shape) #(48, 25, 258) (12, 25, 258) (48, 1) (12, 1)

(192, 25, 258) (48, 25, 258) (192, 4) (48, 4)


<h2><a id='III'></a>
3.  Model Training </h2> 

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [13]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=700, callbacks=[tb_callback])

Epoch 1/700
11/11 [==============================] - 6s 75ms/step - loss: 2.4544 - categorical_accuracy: 0.2690
Epoch 2/700
11/11 [==============================] - 1s 98ms/step - loss: 1.5806 - categorical_accuracy: 0.2281
Epoch 3/700
11/11 [==============================] - 1s 95ms/step - loss: 1.3344 - categorical_accuracy: 0.2836
Epoch 4/700
11/11 [==============================] - 1s 83ms/step - loss: 1.3487 - categorical_accuracy: 0.3509
Epoch 5/700
11/11 [==============================] - 1s 76ms/step - loss: 1.3123 - categorical_accuracy: 0.3363
Epoch 6/700
11/11 [==============================] - 1s 77ms/step - loss: 1.1858 - categorical_accuracy: 0.4181
Epoch 7/700
11/11 [==============================] - 1s 80ms/step - loss: 1.9413 - categorical_accuracy: 0.3509
Epoch 8/700
11/11 [==============================] - 1s 81ms/step - loss: 2.2583 - categorical_accuracy: 0.2573
Epoch 9/700
11/11 [==============================] - 1s 84ms/step - loss: 1.4649 - categorical_accuracy:

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 596,708
Trainable params: 596,708
Non-trai

In [23]:
# Définir le chemin du répertoire des poids
weights_dir = "Weights"

# Assurez-vous que le répertoire existe, sinon, créez-le
if not os.path.exists(weights_dir):
    os.makedirs(weights_dir)

# Sauvegarder les poids dans le fichier 'action.h5' dans le répertoire des poids
model.save(os.path.join(weights_dir, 'action_recognition_9.h5'))

In [22]:
model.load_weights(os.path.join(weights_dir, 'action_recognition.h5'))

### Hand Model Training

In [15]:
model_hands = Sequential()
model_hands.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length,258))) # (25, 258)
model_hands.add(LSTM(128, return_sequences=True, activation='relu'))
model_hands.add(LSTM(64, return_sequences=False, activation='relu'))
model_hands.add(Dense(64, activation='relu'))
model_hands.add(Dense(32, activation='relu'))
model_hands.add(Dense(actions.shape[0], activation='softmax'))

In [16]:
model_hands.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
model_hands.fit(X_train_hands, y_train_hands, epochs=1000, callbacks=[tb_callback])
# (48, 25, 258) (48, 1)


Epoch 1/1000
6/6 [==============================] - 0s 73ms/step - loss: 3.4024e-07 - categorical_accuracy: 1.0000
Epoch 2/1000
6/6 [==============================] - 0s 36ms/step - loss: 3.3838e-07 - categorical_accuracy: 1.0000
Epoch 3/1000
6/6 [==============================] - 0s 38ms/step - loss: 3.3652e-07 - categorical_accuracy: 1.0000
Epoch 4/1000
6/6 [==============================] - 0s 37ms/step - loss: 3.3465e-07 - categorical_accuracy: 1.0000
Epoch 5/1000
6/6 [==============================] - 0s 36ms/step - loss: 3.3341e-07 - categorical_accuracy: 1.0000
Epoch 6/1000
6/6 [==============================] - 0s 39ms/step - loss: 3.3155e-07 - categorical_accuracy: 1.0000
Epoch 7/1000
6/6 [==============================] - 0s 37ms/step - loss: 3.3031e-07 - categorical_accuracy: 1.0000
Epoch 8/1000
6/6 [==============================] - 0s 38ms/step - loss: 3.2969e-07 - categorical_accuracy: 1.0000
Epoch 9/1000
6/6 [==============================] - 0s 37ms/step - loss: 3.2720e

In [23]:
model_hands.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 4)                 132       
                                                                 
Total params: 237,284
Trainable params: 237,284
Non-tr

In [18]:
# Définir le chemin du répertoire des poids
weights_dir_hands = "Weights_hands"


In [23]:
# Sauvargarde des poids
# Assurez-vous que le répertoire existe, sinon, créez-le
if not os.path.exists(weights_dir_hands):
    os.makedirs(weights_dir_hands)

# Sauvegarder les poids dans le fichier 'action.h5' dans le répertoire des poids
model_hands.save(os.path.join(weights_dir_hands, 'action_recognition_5.h5'))

In [19]:
# Chargement des poids
model_hands.load_weights(os.path.join(weights_dir_hands, 'action_recognition_5.h5'))

<h2><a id='IV'></a>
4.  Real time test </h2> 

In [20]:
colors = [(245,117,16), (117,245,16), (16,117,245), (147, 112, 219)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

### Hand + Face

In [11]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
(1662,)


### Real time Hand test

In [21]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints[:258])
        sequence = sequence[-25:]
        
        if len(sequence) == 25:
            res = model_hands.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti